In [ ]:
!pip install torch==1.12.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!python -c "import torch; print(torch.__version__)"

1.12.1+cu113


In [ ]:
%%capture
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
import os
import typing
import torch_geometric

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.datasets as datasets
from torch_geometric.nn import GCNConv, GATConv

import pandas as pd
import warnings
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
dataset = datasets.Planetoid(
    root="./",
    name='CiteSeer',
    split="public",
    transform=torch_geometric.transforms.GCNNorm()
  )

input_dim = dataset.num_features
n_classes = dataset.num_classes

print(dataset.data)

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


Processing...
Done!


In [ ]:
class GAT(nn.Module):
  def __init__(
      self,
      input_dim: int,
      hid_dim: int,
      n_classes: int,
      n_layers: int,
      skip_connections: int,
      dropout_ratio: float = 0.3):
    super(GAT, self).__init__()

    self.input_dim = input_dim
    self.hid_dim = hid_dim
    self.n_classes = n_classes
    self.n_layers = n_layers
    self.skip_connections = skip_connections
    self.dropout_ratio = dropout_ratio

    modules = []
    coefficient_modules = []

    if self.n_layers == 1:
        modules.append(GATConv(self.input_dim, self.n_classes, feat_drop=dropout_ratio))
    else:
        modules.append(GATConv(self.input_dim, self.hid_dim, feat_drop=dropout_ratio))
        for _ in range(self.n_layers - 2):
            coefficient_modules.append(nn.Linear(self.hid_dim, 1))
            modules.append(GATConv(self.hid_dim, self.hid_dim, feat_drop=dropout_ratio))
        modules.append(GATConv(self.hid_dim, self.n_classes, feat_drop=dropout_ratio))

    self.sigmoid = torch.nn.Sigmoid()
    self.softmax = torch.nn.Softmax(dim=1)
    self.layers = torch.nn.ModuleList(modules)
    self.coefficient_layers = torch.nn.ModuleList(coefficient_modules)

  def forward(self, X, A) -> torch.Tensor:
    return self.softmax(self.generate_node_embeddings(X, A))

  def generate_node_embeddings(self, X, A) -> torch.Tensor:

    # No skip connections
    if self.skip_connections == 0:
        output = X
        for i, layer in enumerate(self.layers):
            if i < len(self.layers) - 1:
                output = F.dropout(F.relu(layer(output, A)),
                                   p=self.dropout_ratio,
                                   training=self.training)
            else:
                output = layer(output, A)
        return output

    else:
        output = X
        skip = None
        for i, layer in enumerate(self.layers):

            # first layer
            if i == 0:
                output = F.dropout(F.relu(layer(output, A)),
                                   p=self.dropout_ratio,
                                   training=self.training)
                
                # store first layer for skip connection
                skip = output

            # last layer
            elif i == len(self.layers) - 1:
                output = layer(output, A)

            # hidden layers
            else:
                
                # Normal skip connections
                if self.skip_connections == 1:

                    output = F.dropout(F.relu(layer(output, A)),
                                       p=self.dropout_ratio,
                                       training=self.training) + skip

                # Weighted skip connections
                elif self.skip_connections == 2:

                    weight = F.dropout(self.sigmoid(self.coefficient_layers[i-1](output)),
                                       p=self.dropout_ratio,
                                       training=self.training)
                    output = (1 - weight) * F.dropout(F.relu(layer(output, A)),
                                                      p=self.dropout_ratio,
                                                      training=self.training) + weight * skip

    return output


In [ ]:
def evaluate(
    model,
    data,
    mask
):
    model.eval()
    predictions = torch.argmax(model(data.x, data.edge_index), dim=1)
    return torch.sum(predictions[mask] == data.y[mask]) / torch.sum(mask)

def train(
    params: typing.Dict
) -> torch.nn.Module:

    # set GPU or CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # load dataset
    data = dataset.data
    data = data.to(device)

    model = GAT(
        params["input_dim"], 
        params["hid_dim"],
        params["n_classes"],
        params["n_layers"],
        params["skip_connections"],
        params["dropout"]
    ).to(device)

    optimizer = torch.optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()), 
        lr=params["lr"],
        weight_decay=params["weight_decay"]
    )

    loss_criterion = nn.CrossEntropyLoss()

    model.train()
    optimizer.zero_grad()

    val_accuracies = []

    for epoch in range(params["epochs"]):

        model.train()
        predictions = model(data.x, data.edge_index).to(device)

        loss = loss_criterion(predictions[data.train_mask], torch.nn.functional.one_hot(
            data.y[data.train_mask], params["n_classes"]).float())

        loss.backward()
        optimizer.step()

        val_accuracy = evaluate(model, data, data.val_mask)
        val_accuracies.append(val_accuracy.item())

        if len(val_accuracies) >= params["max_patience"]:
            decreasing = True
            for i in range(1, params["max_patience"]):
                decreasing &= (val_accuracies[-i] < val_accuracies[-i-1])
            if decreasing:
                break  # Early stop

    test_accuracy = evaluate(model, data, data.test_mask).item()

    return model, test_accuracy

In [ ]:
# PARAMETERS

training_params = {
    "lr": 0.005,                # learning rate
    "weight_decay": 0.005,      # weight_decay
    "epochs": 100,              # total training epochs
    "max_patience": 5,          # early stopping time
    "hid_dim": 128,             # hidden layer size
    "n_layers": None,           # number of attention layers
    "skip_connections": 0,      # 0 for no connections, 1 for normal, 2 for weighted connections
    "dropout": 0.5,             # dropout rate
    "input_dim": input_dim,     # input feature size
    "n_classes": n_classes,     # number of classes
}

In [ ]:
torch.manual_seed(123)

# TRAIN VARIOUS MODELS

num_layers = [1, 3, 6, 10, 20]
skip_connections = [0, 1, 2]
N = 10

accuracies = np.zeros((len(skip_connections), len(num_layers)))

for j, num_layer in enumerate(num_layers):
    for i, skip_connection in enumerate(skip_connections):
        
        training_params["skip_connections"] = skip_connection
        training_params["n_layers"] = num_layer

        test_accuracies = []

        for _ in range(0, N):
            model, test_acc = train(training_params)
            test_accuracies += [test_acc]

        print(f"Mean: {np.round(np.mean(test_accuracies), 3)},  Skip connection: {skip_connection},  num_layers: {num_layer}")

        accuracies[i][j] = np.mean(test_accuracies)

print(accuracies)

In [ ]:
def dimension_reduction(model) -> pd.DataFrame:
    model.eval()
    embedding = model.generate_node_embeddings(dataset.data.x, dataset.data.edge_index)[dataset.data.test_mask]
    X_embedded = TSNE(n_components=2).fit_transform(embedding.detach().cpu())
    df = pd.DataFrame(X_embedded)
    df.insert(2, 'labels', dataset.data.y[dataset.data.test_mask].detach().cpu())
    return df

In [ ]:
def visualise(feature_dict: typing.Dict, name: str) -> None:
    colors = {0:'mediumaquamarine', 1:'dodgerblue', 2:'seagreen', 3:'gold', 4:'orangered', 5:'orchid', 6:'teal'}

    plt.figure(figsize=(22, 4))
    
    for i, key in enumerate(feature_dict):
        plt.subplot(1, len(feature_dict), i+1)
        data = feature_dict[key]
        plt.scatter(data[0], data[1], c=data['labels'].map(colors))
        plt.title(key)

    plt.savefig(name + ".png", format = 'png', dpi=500)
    plt.show()

In [ ]:
# VISUALISE EMBEDDINGS

for i in range(0, 3):

    training_params["skip_connections"] = i

    training_params["n_layers"] = 1
    A, _ = train(training_params)

    training_params["n_layers"] = 3
    B, _ = train(training_params)

    training_params["n_layers"] = 7
    C, _ = train(training_params)

    training_params["n_layers"] = 10
    D, _ = train(training_params)

    training_params["n_layers"] = 20
    E, _ = train(training_params)

    feature_dict = {
        "1 layer": dimension_reduction(A),
        "3 layers": dimension_reduction(B),
        "7 layers": dimension_reduction(C),
        "10 layers": dimension_reduction(D),
        "20 layers": dimension_reduction(E),
    }

    visualise(feature_dict, f"skip_connection_{i}")